<a href="https://colab.research.google.com/github/guilhermeterenciani/IA/blob/main/14_tokenizadores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pandas as pd
from nltk.tokenize import word_tokenize,sent_tokenize
import pickle
import nltk

import os

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
os.chdir('/content/drive/MyDrive/ml-nlp/')

In [ ]:
!gdown https://drive.google.com/uc?id=1-XXx9A7EMCfP5LLLPLk6HIJbaJLDGvNa

Downloading...
From: https://drive.google.com/uc?id=1-XXx9A7EMCfP5LLLPLk6HIJbaJLDGvNa
To: /content/drive/MyDrive/ml-nlp/dataset.p
100% 20.7M/20.7M [00:00<00:00, 166MB/s]


In [ ]:
corpus = pickle.load(open('dataset.p','rb'))
scorpus = []
for inst in corpus:
    scorpus += sent_tokenize(inst['body'])
    scorpus += sent_tokenize(inst['title'])

fcorpus = []
for inst in scorpus:
    fcorpus += [word_tokenize(inst)]

#Tokens especiais (twitter)

In [ ]:
!pip install ekphrasis

In [ ]:
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
        'time', 'url', 'date', 'number'],
    # terms that will be annotated
    annotate={"hashtag", "allcaps", "elongated", "repeated",
        'emphasis', 'censored'},
    fix_html=True,  # fix HTML tokens

    # corpus from which the word statistics are going to be used
    # for word segmentation
    segmenter="twitter",

    # corpus from which the word statistics are going to be used
    # for spell correction
    corrector="twitter",

    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=False,  # spell correction for elongated words

    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,

    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)

/usr/local/lib/python3.7/dist-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Reading twitter - 1grams ...
Reading twitter - 2grams ...
Reading twitter - 1grams ...


/usr/local/lib/python3.7/dist-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


In [ ]:
text_processor.pre_process_doc("hj esta 🔥 😁 :)")

['hj', 'esta', '🔥', '😁', '<happy>']

# Fasttext

In [ ]:
from gensim.models import FastText

In [ ]:
sentences = [["cat", "say", "meow"], ["dog", "say", "woof","faca","comp"]]
model = FastText(sentences, min_count=1)

In [ ]:
model.most_similar("facom")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('faca', 0.5361242294311523),
 ('comp', 0.25643062591552734),
 ('dog', 0.20705343782901764),
 ('meow', 0.09984646737575531),
 ('say', 0.056755296885967255),
 ('cat', -0.06016944348812103),
 ('woof', -0.06544774025678635)]

#Byte-Pair Encoding

In [ ]:
!pip install tokenizers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from tokenizers import Tokenizer
from tokenizers.models import BPE

tokenizer = Tokenizer(BPE())

from tokenizers.pre_tokenizers import Whitespace

tokenizer.pre_tokenizer = Whitespace()

In [ ]:
from tqdm.auto import tqdm

text_data = []
file_count = 0

for sample in tqdm(scorpus):
    # remove newline characters from each sample as we need to use exclusively as seperators
    sample = sample.replace('\n', '\s')
    text_data.append(sample)
    if len(text_data) == 5_000:
        # once we hit the 5K mark, save to file
        with open(f'./text_{file_count}.txt', 'w', encoding='utf-8') as fp:
            fp.write('\n'.join(text_data))
        text_data = []
        file_count += 1
# after saving in 5K chunks, we may have leftover samples, we save those now too
with open(f'./text_{file_count}.txt', 'w', encoding='utf-8') as fp:
    fp.write('\n'.join(text_data))

  0%|          | 0/95435 [00:00<?, ?it/s]

In [ ]:
from pathlib import Path
paths = [str(x) for x in Path('./').glob('**/*.txt')]
paths

['bert-base-uncased-vocab.txt',
 'bert-base-cased-vocab.txt',
 'text_9.txt',
 'text_7.txt',
 'text_6.txt',
 'text_3.txt',
 'text_0.txt',
 'text_2.txt',
 'text_5.txt',
 'text_4.txt',
 'text_1.txt',
 'text_8.txt',
 'text_10.txt',
 'text_13.txt',
 'text_16.txt',
 'text_15.txt',
 'text_11.txt',
 'text_12.txt',
 'text_14.txt',
 'text_17.txt',
 'text_19.txt',
 'text_18.txt']

In [ ]:
from tokenizers.trainers import BpeTrainer

In [ ]:
trainer = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])

In [ ]:
tokenizer.train(paths, trainer=trainer)

In [ ]:
enc= tokenizer.encode('computação faculdade de computação FACOM inflação')

In [ ]:
enc.tokens

['computação', 'faculdade', 'de', 'computação', 'FAC', 'OM', 'inflação']

#WordPiece

In [ ]:
!wget https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt

--2023-03-01 21:34:50--  https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.201.248, 54.231.131.104, 54.231.203.112, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.201.248|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 213450 (208K) [text/plain]
Saving to: ‘bert-base-cased-vocab.txt.2’

bert-base-cased-voc 100%[===================>] 208.45K  --.-KB/s    in 0.09s   

2023-03-01 21:34:50 (2.34 MB/s) - ‘bert-base-cased-vocab.txt.2’ saved [213450/213450]



In [ ]:
from tokenizers import BertWordPieceTokenizer

sequence = "How are you Tokenizer 😁 ?"
tokenizerBW = BertWordPieceTokenizer("bert-base-cased-vocab.txt")
tokenized_sequenceBW = tokenizerBW.encode(sequence)
print(tokenized_sequenceBW)
print(type(tokenized_sequenceBW))
print(tokenized_sequenceBW.ids)
print(tokenized_sequenceBW.tokens)

Encoding(num_tokens=9, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
<class 'tokenizers.Encoding'>
[101, 1293, 1132, 1128, 22559, 17260, 100, 136, 102]
['[CLS]', 'how', 'are', 'you', 'token', '##izer', '[UNK]', '?', '[SEP]']


In [ ]:
tokenized_sequenceBW = tokenizerBW.encode("surfboarded surf Surfing surfed computing")
print(tokenized_sequenceBW)
print(type(tokenized_sequenceBW))
print(tokenized_sequenceBW.ids)
print(tokenized_sequenceBW.tokens)

Encoding(num_tokens=10, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
<class 'tokenizers.Encoding'>
[101, 20114, 4015, 1174, 20114, 25502, 20114, 1174, 12783, 102]
['[CLS]', 'surf', '##board', '##ed', 'surf', 'surfing', 'surf', '##ed', 'computing', '[SEP]']
